In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from pandas import DataFrame
users = dict()
items = dict()
rows = []
cols = []
data = []
i = 0
j = 0
for l in open("drive/MyDrive/추천2/train.json", 'r'):
  d = eval(l)
  u = d['userID']
  g = d['gameID']
  rating = d['hours_transformed']
  if u in users:
    ui = users[u]
  else:
    users[u] = i
    ui = i
    i += 1
  if g in items:
    gi = items[g]
  else:
    items[g] = j
    gi = j
    j += 1
  rows.append(ui)
  cols.append(gi)
  data.append(rating)

rows = np.array(rows)
cols = np.array(cols)
train = dict()
train['userID'] = rows
train['gameID'] = cols
train = DataFrame(train)
train['count'] = data

with open("refined_train.csv", "w") as f2:
  f2.write("userID,gameID,hours\n")
  for i , j , k in zip(rows, cols , data):
    f2.write(f"{i},{j} , {k}\n")

In [ ]:
with open("drive/MyDrive/추천2/pairs_Hours.csv", "r") as f1:
    f1.readline()
    lines = []
    uids = []
    gids = []
    g_cs = []
    u_cs = []
    p_ts = []
    test = dict()
    for line in f1:
        row_id, uid, gid = line.strip().split(",")
        lines.append(row_id)
        uids.append(users[uid])
        gids.append(items[gid])
    with open("Hours.csv", "w") as f2:
        f2.write("ID,userID,gameID\n")
        for a , b , c in zip(lines , uids , gids):
            f2.write(f"{a},{b},{c}\n")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

df = pd.read_csv('refined_train.csv')

X = df[['userID', 'gameID']]
y = df['hours']
transformer = ColumnTransformer([('onehot', OneHotEncoder(handle_unknown='ignore' , sparse_output=True), ['userID', 'gameID'])], remainder='passthrough')
X_onehot = transformer.fit_transform(X)
# X_train_onehot, X_test_onehot, y_train, y_test = train_test_split(X_onehot, y, test_size=0.2, random_state=42)
model = Ridge(alpha=6 , )
model.fit(X_onehot, y)
# y_pred_onehot = model.predict(X_test_onehot)
# mse_onehot = mean_squared_error(y_test, y_pred_onehot)
# print(mse_onehot)
# print(model_onehot.coef_)
# print(model_onehot.intercept_)

test = pd.read_csv("Hours.csv")

X_test = test[['userID', 'gameID']]
X_onehot_test = transformer.transform(X_test)
y_pred_test = model.predict(X_onehot_test)
with open("Hours.csv", "r") as f1:
  f1.readline()
  with open("result.csv", "w") as f2:
    f2.write("ID,Label\n")
    for line , h in zip(f1,y_pred_test):
      h = max(0 , h)
      row_id, uid, gid = line.strip().split(",")
      f2.write(f"{row_id},{h}\n")
